In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [2]:
browser = webdriver.Chrome()

In [3]:
url = "https://www.29cm.co.kr/shop/category/list?category_large_code=272100100&category_medium_code=272102100&sort=order&category_small_code=&page=1&brand=&min_price=&max_price=&is_free_shipping=&is_discount=&is_soldout=&colors=&count=50"
browser.get(url)
time.sleep(1)

In [4]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [9]:
results = []
current_page = 2
max_retries = 3  # 최대 재시도 횟수
rank = 1

timeout_threshold = 60
# 마지막으로 성공적인 페이지 스크래핑 시간을 추적하는 변수
last_successful_time = time.time()

total_page = soup.select('div.product_content.ng-star-inserted > ruler-basic-pagination > div > span > a.ng-star-inserted')[8].text.strip()
while current_page < int(total_page):
    retries = 0
    if current_page == 2:
        soup = BeautifulSoup(browser.page_source, "html.parser")
        item_list = soup.select('li > ruler-product-list-large-item > div.prd_b')
        time.sleep(1)
        
        for item in item_list:

                brand_list = item.select('div.info > div.brand')[0].text.replace('\n','').strip()
                raw_title = item.select('div.info > div.name')[0].text.replace('\n','').strip()
                img_list = item.select('div.imgbx > div.imgin > ruler-blazy > img')[0]['src']
                link_list = item.select('a.prd_b_area')[0]['href']

                try :
                    price_original = item.select('div.info > div.price > div > ruler-price-text > span.price_sale')[0].text.replace('\n','').strip()
                    price_now = item.select('div.info > div.price > div > ruler-price-text > span.price_sale')[1].text.replace('\n','').strip()
                except Exception as e:
                    price_list = item.select('div.info > div.price')[0].text.replace('\n','').strip()                   
                    split_price = price_list.split()
                    if len(split_price)>= 2 :
                        price_original = split_price[0]+'원'
                        price_now = split_price[1]
                    else :
                        price_original = split_price[0]
                        price_now = split_price[0]

                if raw_title.startswith('('):
                    title_list = raw_title.split(')', 1)[1].strip()
                elif raw_title.startswith('['):
                    title_list = raw_title.split(']', 1)[1].strip()
                elif '배송' in raw_title:
                    title_list = raw_title.split('송', 1)[1].strip()
                else:
                    title_list = raw_title
        #       print(brand_list, title_list ,price_original ,price_now,img_list)
                data = [rank, brand_list, title_list ,price_original ,price_now ,link_list ,img_list]
                results.append(data)
                rank += 1
                last_successful_time = time.time()
                
        current_page += 1 
        print(current_page)
    else : 
        while retries < max_retries:
            try:      
                soup = BeautifulSoup(browser.page_source, "html.parser")
                item_list = soup.select('li > ruler-product-list-large-item > div.prd_b')
                time.sleep(1)
                
                if current_page > 1:  # 첫 페이지가 아닌 경우에만 페이지 이동
                    page_btn = browser.find_elements('css selector', 'a > ruler-svg-icon-next > svg')[0]
                    page_btn.click()
                    time.sleep(2)
                    
                for item in item_list:

                    brand_list = item.select('div.info > div.brand')[0].text.replace('\n','').strip()
                    raw_title = item.select('div.info > div.name')[0].text.replace('\n','').strip()
                    img_list = item.select('div.imgbx > div.imgin > ruler-blazy > img')[0]['src']
                    link_list = item.select('a.prd_b_area')[0]['href']

                    try :
                        price_original = item.select('div.info > div.price > div > ruler-price-text > span.price_sale')[0].text.replace('\n','').strip()
                        price_now = item.select('div.info > div.price > div > ruler-price-text > span.price_sale')[1].text.replace('\n','').strip()
                    except Exception as e:
                        price_list = item.select('div.info > div.price')[0].text.replace('\n','').strip()                   
                        split_price = price_list.split()
                        if len(split_price)>= 2 :
                            price_original = split_price[0]+'원'
                            price_now = split_price[1]
                        else :
                            price_original = split_price[0]
                            price_now = split_price[0]

                    if raw_title.startswith('('):
                        title_list = raw_title.split(')', 1)[1].strip()
                    elif raw_title.startswith('['):
                        title_list = raw_title.split(']', 1)[1].strip()
                    elif '배송' in raw_title:
                        title_list = raw_title.split('송', 1)[1].strip()
                    else:
                        title_list = raw_title
            #       print(brand_list, title_list ,price_original ,price_now,img_list)
                    data = [rank, brand_list, title_list, price_original, price_now, link_list, img_list]
                    results.append(data)
                    rank += 1 
                    last_successful_time = time.time()
                    
                current_page += 1
                print(current_page)
                break

            except Exception as e:
                print(f"Error occurred on page {current_page}: {e}, retrying...")
                retries += 1
                time.sleep(5)  # 에러 발생 시 잠시 기다린 후 재시도

        if retries == max_retries:
            print(f"Max retries reached on page {current_page}. Moving to next page.")
            current_page += 1

        if current_page >= int(total_page):
            break  # 전체 페이지 처리 완료 후, 외부 while 루프 종료
            
        if time.time() - last_successful_time > timeout_threshold:
            print("타임아웃에 도달했습니다. 루프를 종료합니다.")
            break
    # 결과 출력 또는 저장
print(results)


In [8]:
import pandas as pd
df = pd.DataFrame(results)
df.columns = ['랭킹', '브랜드', '상품명', '기존가격', '현재가격', '사이트' ,'이미지링크']
df.to_excel('./data/29cm_outer.xlsx',index = False)